## Setup

In [1]:
import sys

sys.path.append("../")

In [2]:
import os
from ast import literal_eval

import cartopy.crs as ccrs
import dask
import dask.config
import geopandas as gpd
import matplotlib.colors as mcols
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib.colors import LinearSegmentedColormap
from pyCIAM.constants import COSTTYPES
from shared import (
    DIR_FIGS,
    PATH_BORDERS,
    PATH_COASTLINES,
    PATH_DIAZ_INPUTS_INT,
    PATH_DIAZ_RES,
    PATH_GADM,
    PATH_MOVEFACTOR_DATA,
    PATH_OUTPUTS,
    PATH_PARAMS,
    PATH_PARAMS_DIAZ,
    PATH_PWT,
    PATH_SLIIDERS,
    PATH_SLIIDERS_INCOME_INTERMEDIATE_FILE,
    PATH_SLR_AR5_QUANTILES,
    PATH_SLR_AR6,
    PATH_SLR_SWEET,
    QUANTILES,
    open_zarr,
    read_shapefile,
    start_dask_cluster,
)

# SLR scenarios modeled but to exclude from analysis
drop_scens = ["ssp245_low"]

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [7]:
PARAMS = pd.read_json(str(PATH_PARAMS))["values"]
PARAMS_DIAZ = pd.read_json(str(PATH_PARAMS_DIAZ))["values"]

# Selected constants for plotting
CASES = ["optimalfixed", "noAdaptation"]

RESENDYR = 2100  # last year of results to consider
POPWTYR = 2000  # year of population to use in % pop adaptation strategy plots
PPP_BASELINE_YEAR = 2019  # dollar value year of results

# When running on larger/scalable dask cluster, may wish to specify number of workers
# Default is LocalCluster which will use the number of CPUs available on local machine
N_WORKERS = None
# N_WORKERS = 128

# max elev in Diaz, and max to consider in % pop and length comparisons w pyCIAM
MAXELEV = 15

# pyCIAM SLR, IAM, SSP scenarios to focus on for global maps
NCC_SCEN = ("AR6", "NCC")
SCEN = ("AR6-Medium", "SSP2-4.5")
IAM = "IIASA"
SSP = "SSP2"

# color-blind robust palette for scatter plots
cols = ["#CC79A7", "#D55E00", "#0072B2", "#E8C602", "#50CAA8", "#56B4E9", "#E69F00"]

## Start Dask Cluster

In [8]:
client = start_dask_cluster(n_workers=N_WORKERS)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/daskhub-dev.61b117f50a8647b39d280c62a01865fc/status,


## Helper funcs

In [19]:
def rename_pc(val):
    split = val.split("_")
    if len(split) == 1:
        cat = "Sweet"
        scen = split[0]
    else:
        split = ["_".join(split[:-1]), split[-1]]
        if "190726" in split[0]:
            cat = "B19"
            scen = split[1]
        elif split[1].startswith("rcp"):
            scen = "RCP " + split[1][-2] + "." + split[1][-1]
            if "170113" in split[0]:
                cat = "K14"
            elif "190726" in split[0]:
                cat = "B19"
            elif "200204" in split[0]:
                cat = "D21"
            elif "210628" in split[0]:
                cat = "SROCC"
        elif split[0] == "ncc":
            if split[1].startswith("ar"):
                cat = split[1].upper()
            else:
                cat = split[1].title()
            scen = split[0].upper()
        elif split[0].startswith("ssp"):
            scen = split[0][:4].upper() + "-" + split[0][4] + "." + split[0][5:]
            cat = "AR6-" + split[1].title()

    return (cat, scen)


def rename_d16(scen_val):
    if scen_val == "ncc":
        return "NCC"
    else:
        return f"RCP {int(scen_val[3:]) / 10:.1f}"


def ds_filt(
    pyCIAM_da,
    diaz_da,
    case="optimalfixed",
    costtypes=COSTTYPES,
    cconly=False,
):
    """Filter pyCIAM and Diaz2016 results datasets by variable, case, scenarios, years
    (used for plotting results subsets)

    Parameters
    ----------
    pyCIAM_ds : xr.Dataset
        dataset of pyCIAM results to be filtered
    diaz_ds : xr.Dataset
        dataset of pyCIAM (configured in Diaz2016 mode) results to be filtered
    val : str
        name of data variable to be selected in filtered output ("npv", "costs")
    case : str
        name of model case to be selected ("optimalfixed","noAdaptation")
    costtypes : list of str
        cost types to be selected ("inundation", "protection", ...)
    cconly : True/False
        conditional to select just climate-change scenarios (True) or all (False)
    pyCIAM_quantiles : list
        SLR quantiles of pyCIAM results to consider

    Returns
    -------
    two xr.DataArray objects representing filtered results from input Datasets

    """

    diaz_da = diaz_da.sel(quantile=0.5)
    pyCIAM_da = pyCIAM_da.sel(quantile=0.5)

    if cconly:
        pyCIAM_da = pyCIAM_da.sel(scenario=pyCIAM_da.scen != "NCC")
        diaz_da = diaz_da.sel(scenario=diaz_da.scen != "NCC")

    selectors = {
        k: v
        for k, v in [
            ("case", case),
            ("costtype", costtypes),
        ]
        if k in pyCIAM_da.dims
    }
    sum_dims = [d for d in ["costtype"] if d in selectors.keys()]

    return [(da.sel(selectors).sum(sum_dims)) for da in [pyCIAM_da, diaz_da]]

## Import Data

### SLR

In [20]:
gmsl_da = xr.concat(
    [
        open_zarr(p).gsl_msl05.sel(year=2100, quantile=QUANTILES).drop("year").load()
        for p in (PATH_SLR_AR5_QUANTILES, PATH_SLR_AR6, PATH_SLR_SWEET)
    ],
    dim="scenario",
).drop_sel(scenario="ssp245_low")

scen_names = gmsl_da.scenario.to_series().apply(rename_pc)
gmsl_da = gmsl_da.assign_coords(
    ID=scen_names.str[0], scen=scen_names.str[1]
).set_xindex(["ID", "scen"])
gmsl = (
    gmsl_da.to_series().unstack("quantile").reorder_levels(["ID", "scen"]).sort_index()
)

# AR6-Medium and Sweet are the colored in scenarios for scatter plots
pc_sel_scens = gmsl.loc[pd.IndexSlice[["AR6-Medium", "Sweet"]]].index.values

In [21]:
# add epsilon to make sure .5 gets rounded up (allowing for floating point precision)
(gmsl + 1e-10).round(2)

quantile                0.17  0.50  0.83
ID         scen                         
AR6-Low    SSP1-2.6     0.32  0.45  0.79
           SSP5-8.5     0.63  0.88  1.61
AR6-Medium SSP1-1.9     0.28  0.38  0.55
           SSP1-2.6     0.32  0.44  0.61
           SSP2-4.5     0.44  0.56  0.76
           SSP3-7.0     0.55  0.68  0.90
           SSP5-8.5     0.63  0.77  1.02
B19        2p0degree+L  0.48  0.68  0.96
           rcp85+H      0.79  1.10  1.71
D21        RCP 2.6      0.43  0.52  0.61
           RCP 4.5      0.52  0.62  0.74
           RCP 8.5      0.90  1.10  1.32
K14        RCP 2.6      0.35  0.48  0.65
           RCP 4.5      0.43  0.58  0.76
           RCP 8.5      0.61  0.78  1.00
SROCC      RCP 2.6      0.39  0.49  0.60
           RCP 4.5      0.48  0.60  0.76
           RCP 8.5      0.71  0.88  1.11
Sweet      High         1.95  1.98  2.02
           Int          0.98  0.98  0.99
           IntHigh      1.47  1.48  1.50
           IntLow       0.48  0.48  0.49
           Low          0.28  0.28  0.29

### pyCIAM/Diaz Inputs and Results

Load and harmonize scenario names

In [22]:
# pyCIAM inputs
inputs = open_zarr(PATH_SLIIDERS, chunks=None)

# diaz16 inputs
d16inputs = open_zarr(PATH_DIAZ_INPUTS_INT, chunks=None)

# pyCIAM results
with dask.config.set(**{"array.slicing.split_large_chunks": False}):
    res = (
        open_zarr(PATH_OUTPUTS)
        .sel(case=CASES, quantile=QUANTILES)
        .drop_sel(scenario=drop_scens)
        .persist()
    )

# Diaz 2016 results ($2010)
d16 = open_zarr(PATH_DIAZ_RES)

# harmonize scenario names
scen_names = res.scenario.to_series().apply(rename_pc)
res = res.assign_coords(ID=scen_names.str[0], scen=scen_names.str[1]).set_xindex(
    ["ID", "scen"]
)

d16 = (
    d16.assign_coords(scenario=d16.scenario.to_series().apply(rename_d16))
    .rename(scenario="scen")
    .expand_dims(ID=["K14"])
    .stack(scenario=["ID", "scen"])
)

# add Diaz NPV consistent with original paper
d16["npv"] = (
    (
        10
        * d16.costs
        * ((1 / (1 + PARAMS_DIAZ.dr)) ** (d16.year - PARAMS_DIAZ.npv_start)).clip(max=1)
    )
    .sel(year=slice(PARAMS_DIAZ.npv_start, RESENDYR))
    .sum("year")
)

# Add NPV to Diaz dataset that is consistent with pyCIAM (2005-2100)
with dask.config.set(**{"array.slicing.split_large_chunks": False}):
    d16a = (
        d16.reindex(year=np.arange(d16.year.min().item(), RESENDYR + 1))
        .reset_index("scenario")
        .ffill("year")
        .set_xindex(["ID", "scen"])
    )
    d16["npv_consistent"] = (
        d16a.costs
        * ((1 / (1 + PARAMS.dr)) ** (d16a.year - PARAMS.npv_start))
        .clip(max=1)
        .sel(year=slice(PARAMS.npv_start, RESENDYR))
    ).sum("year")
d16 = d16.load()

# add NPV to pyCIAM results
res["npv"] = (
    (res.costs * ((1 / (1 + PARAMS.dr)) ** (res.year - PARAMS.npv_start)).clip(max=1))
    .sel(year=slice(PARAMS.npv_start, RESENDYR))
    .sum("year")
    .persist()
)

In [23]:
mf = open_zarr(PATH_MOVEFACTOR_DATA, chunks=None)
mfdf = mf.sum("seg").to_dataframe()

# Convert to desired units
mfdf["K_2019"] = mfdf.K_2019 / 1e12  # trillions
mfdf["pop_2019"] = mfdf.pop_2019 / 1e6  # millions

# Find midpoint values of K and pop
# y vals
mid = mfdf.max() / 2

xint = np.abs(1 - mfdf / mid).idxmin()
xmid = xint.mean()

In [ ]:
lw = 4  # linewidth
ms = 60  # marker size
mc = "grey"  # marker color
mt = "o"  # marker type

fig, ax = plt.subplots(1, 1, figsize=(18, 10), dpi=300)
ax2 = ax.twinx()
ax.xaxis.set_tick_params(labelsize=25)
ax.yaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)

ax.plot(mfdf.K_2019, lw=lw, c="dodgerblue", zorder=1)
ax2.plot(mfdf.pop_2019, lw=lw, c="coral", zorder=1)

# Midpoint line (same height as popmid)
ax.axhline(mid.K_2019, linestyle="--", lw=2, c="k", alpha=0.2, zorder=3)

# Line intersections
ax.scatter([xint.K_2019], [mid.K_2019], marker=mt, s=ms, zorder=4, c=mc)
ax2.scatter([xint.pop_2019], [mid.pop_2019], s=ms, zorder=4, c=mc)
ax.grid(alpha=0.3, linestyle="--")

# Midpoint of intersections
ax.scatter([xmid], 0, marker="D", c="r", s=80, alpha=0.5)
ax.scatter([xmid], mid.K_2019, marker="x", c="r", s=80, alpha=0.5)
ax.vlines(xmid, 0, mid.K_2019, linestyles="--", colors="r", alpha=0.5)

# Align y-axes
ax.set_ylim(0, None)
ax2.set_ylim(0, None)
ax.set_xlabel("Move Factor", fontsize=20)
ax.set_ylabel("Capital [Trillion $2019]", fontsize=20)
ax2.set_ylabel("Population in 2019 [Million]", fontsize=20)

out_path = DIR_FIGS / "movefactor_K_pop.png"
if os.path.isfile(out_path):
    os.remove(out_path)
plt.savefig(out_path, dpi=300)

## pyCIAM (CIAM config) Diaz16 paper results comp

### Global Values

In [25]:
d16_85 = d16.sel(scenario=("K14", "RCP 8.5"), quantile=0.5)
d16_85_opt = d16_85.sel(case="optimalfixed")

# Global NPV (2010-2100) [$Tn]
d16_85_opt.npv.sum()

<xarray.DataArray 'npv' ()>
array(1.69224573e+12)
Coordinates:
    case      <U12 'optimalfixed'
    quantile  float64 0.5
    scenario  object ('K14', 'RCP 8.5')
    ID        <U3 'K14'
    scen      <U7 'RCP 8.5'

In [26]:
# Wetland Costs in 2100 [$Bn] (annualizing the decadal costs)
d16_85_opt.costs.sel(year=RESENDYR, costtype="wetland").sum().item() / 1e9

79.29315740954944

In [27]:
# Global Costs in 2100 (optimal adapt) [$Bn]
d16_85_opt.costs.sel(year=RESENDYR).sum().item() / 1e9

282.0671058445753

In [28]:
# Global Costs in 2100 (no adapt) [$Tn]
d16_85.costs.sel(year=RESENDYR, case="noAdaptation").sum().item() / 1e12

2.251455307220169

### Country NPV values

In [29]:
country_npv = (
    d16_85_opt.npv.sum("costtype")
    .groupby(d16_85_opt.seg.str.replace("\d+", "").rename("country"))  # noqa: E261
    .sum()
    / 1e9
)
country_npv.sel(country=["UnitedStates", "Brazil", "Australia"])

<xarray.DataArray 'npv' (country: 3)>
array([419.73063722,  97.4954307 , 208.55586383])
Coordinates:
    quantile  float64 0.5
    scenario  object ('K14', 'RCP 8.5')
    ID        <U3 'K14'
    scen      <U7 'RCP 8.5'
    case      <U12 'optimalfixed'
  * country   (country) object 'UnitedStates' 'Brazil' 'Australia'

In [30]:
country_npv.sel(country="China") - country_npv.sel(country="Macau", drop=True)

<xarray.DataArray 'npv' ()>
array(86.99817109)
Coordinates:
    quantile  float64 0.5
    scenario  object ('K14', 'RCP 8.5')
    ID        <U3 'K14'
    scen      <U7 'RCP 8.5'
    case      <U12 'optimalfixed'
    country   <U5 'China'

## Inflation Adjust Diaz, Calc NPV, CC-only Values

In [31]:
# adjust to 2019 USD from 2010 USD
pl_data = (
    pd.read_parquet(
        PATH_PWT,
        columns=["year", "pl_gdpo"],
        filters=[
            ("year", "in", [2010, PPP_BASELINE_YEAR]),
            ("countrycode", "=", "USA"),
        ],
    )
    .set_index("year")
    .pl_gdpo
)

pl_multiplier = pl_data[PPP_BASELINE_YEAR] / pl_data[2010]

# Inflation-adjust Diaz 2016 values to $2019
d16[["costs", "npv", "npv_consistent"]] *= pl_multiplier

In [32]:
# Create CC-only Costs
with dask.config.set(**{"array.slicing.split_large_chunks": False}):
    res_cc_ar5 = res.sel(
        scenario=res.ID.isin(["K14", "B19", "D21", "SROCC"])
    ) - res.sel(scenario=("AR5", "NCC"))

    res_cc_ar6 = res.sel(scenario=res.ID.str.startswith("AR6-")) - res.sel(
        scenario=("AR6", "NCC")
    )

    res_cc_sweet = res.sel(
        scenario=(res.ID == "Sweet") & (res.scen != "NCC")
    ) - res.sel(scenario=("Sweet", "NCC"))

res_cc = xr.concat([res_cc_ar5, res_cc_ar6, res_cc_sweet], dim="scenario").persist()
d16_cc = d16.sel(scenario=d16.scen != "NCC") - d16.sel(scenario=("K14", "NCC"))

## Print Values for Abstract & Intro

In [33]:
# RCP4.5 scenarios
r45 = [i for i in res.scenario.values if "4.5" in i[1]]  # includes AR6, Sweet
print(f"RCP4.5 Scens:\n{r45}")

# high end scenarios
highend = [("Sweet", i) for i in ["IntHigh", "High"]]

# AR6 2-degree (GMSL 0.40-0.69m)
s1 = [
    ("AR6-Low", "SSP1-2.6"),
    ("AR6-Medium", "SSP1-2.6"),
    ("AR6-Medium", "SSP2-4.5"),
    ("AR6-Medium", "SSP3-7.0"),
]

# AR6 4-degree (GMSL 0.58-0.91m)
s2 = [("AR6-Medium", "SSP3-7.0"), ("AR6-Low", "SSP5-8.5"), ("AR6-Medium", "SSP5-8.5")]

RCP4.5 Scens:
[('K14', 'RCP 4.5'), ('D21', 'RCP 4.5'), ('SROCC', 'RCP 4.5'), ('AR6-Medium', 'SSP2-4.5')]


In [34]:
res_sums = (
    res_cc.sel(case=["optimalfixed", "noAdaptation"])
    .sum(["costtype", "adm1"])
    .persist()
)

### Annual Costs in 2100, (median) AR6 2-degree, 4-degree, High-End

In [35]:
this_sums = (
    res_sums.costs.sel(
        scenario=pd.Series(s1 + s2 + highend).drop_duplicates().values,
        quantile=0.5,
        year=RESENDYR,
    ).load()
    / 1e9
)
for ss in [s1, s2, highend]:
    tmp = this_sums.sel(scenario=ss)
    print(f"Scenarios: {ss}")
    print(tmp.min(["scenario", "ssp", "iam"]).values)
    print(tmp.max(["scenario", "ssp", "iam"]).values)
    print("")

Scenarios: [('AR6-Low', 'SSP1-2.6'), ('AR6-Medium', 'SSP1-2.6'), ('AR6-Medium', 'SSP2-4.5'), ('AR6-Medium', 'SSP3-7.0')]
[ 109.485115 1214.219   ]
[ 527.89545 3995.419  ]

Scenarios: [('AR6-Medium', 'SSP3-7.0'), ('AR6-Low', 'SSP5-8.5'), ('AR6-Medium', 'SSP5-8.5')]
[ 196.36949 3113.6511 ]
[ 752.424 6940.88 ]

Scenarios: [('Sweet', 'IntHigh'), ('Sweet', 'High')]
[  421.3369 11220.931 ]
[ 1459.5653 30602.799 ]



### Normalize by GDP

This section involves pre-calculated annualized GDP estimates in $2019 USD PPP from the intermediate steps of SLIIDERS.

In [36]:
exp = open_zarr(PATH_SLIIDERS_INCOME_INTERMEDIATE_FILE)
global_gdp = (exp.gdppc * exp["pop"]).load().sum("ccode")

In [37]:
global_gdp_npv = (
    (global_gdp / (1 + 0.04) ** (global_gdp.year - PARAMS.npv_start))
    .sel(year=slice(PARAMS.npv_start, RESENDYR))
    .sum("year")
)
global_gdp_npv

<xarray.DataArray (ssp: 5, iam: 2)>
array([[4.74688408e+15, 5.16870668e+15],
       [4.56611040e+15, 4.62183058e+15],
       [3.88760430e+15, 3.86227570e+15],
       [3.92412808e+15, 4.33656860e+15],
       [5.57369355e+15, 6.33169986e+15]])
Coordinates:
  * iam      (iam) <U5 'IIASA' 'OECD'
  * ssp      (ssp) <U4 'SSP1' 'SSP2' 'SSP3' 'SSP4' 'SSP5'

In [38]:
global_gdp_2100 = global_gdp.sel(year=2100, drop=True)
global_gdp_2100

<xarray.DataArray (ssp: 5, iam: 2)>
array([[5.28759391e+14, 7.29302505e+14],
       [5.27775926e+14, 6.94511079e+14],
       [3.40489698e+14, 3.59602289e+14],
       [3.02357495e+14, 4.55713678e+14],
       [9.49600984e+14, 1.31006574e+15]])
Coordinates:
  * iam      (iam) <U5 'IIASA' 'OECD'
  * ssp      (ssp) <U4 'SSP1' 'SSP2' 'SSP3' 'SSP4' 'SSP5'

In [39]:
this_rats = this_sums / (global_gdp_2100 / 1e9) * 100
for ss in [s1, s2, highend]:
    tmp = this_rats.sel(scenario=ss)
    print(f"Scenarios: {ss}")
    print(tmp.min(["scenario", "ssp", "iam"]).values)
    print(tmp.max(["scenario", "ssp", "iam"]).values)
    print("")

Scenarios: [('AR6-Low', 'SSP1-2.6'), ('AR6-Medium', 'SSP1-2.6'), ('AR6-Medium', 'SSP2-4.5'), ('AR6-Medium', 'SSP3-7.0')]
[0.02071204 0.11144393]
[0.06494613 1.15788926]

Scenarios: [('AR6-Medium', 'SSP3-7.0'), ('AR6-Low', 'SSP5-8.5'), ('AR6-Medium', 'SSP5-8.5')]
[0.04029534 0.29192208]
[0.08897142 1.97495206]

Scenarios: [('Sweet', 'IntHigh'), ('Sweet', 'High')]
[0.08306414 1.24713468]
[0.19734302 7.60298276]



## Compare Diaz 2016 reported values to pyCIAM (Scatter Plots) - All Median Values

In [40]:
def compPlot(
    pc,
    dc,
    col_scens=pc_sel_scens,
    xlab=None,
    ylab=None,
    title=None,
    filename="myplot",
    xmin=-0.05,
    xmax=np.nan,
    ymin=0,
    ymax=np.nan,
    cvals=cols[0:5],
    x_iam_jitter=0.01,
    plot_trend=True,
    trend_poly=1,
    y_norm=1e12,
    dpi=300,
):
    """
    Parameters
    ----------
    col_scens : list
        the selected scenarios to plot with colored markers
    xlab, ylab, title : str
        label for x, y, title axis values
    filename : str
        name of output graph file (.png)
    xmin, xmax, ymin, ymax : float
        figure axes ranges
    cvals : list
        hex color values to use for SSP colors
    x_iam_jitter : float
        x-axis distance to jitter (L/R) scatter markers by IAM
    plot_trend : True/False
        Boolean flag for including trend lines on plots. Default True
    trend_poly : 1
        Polynomial order for which to fit trend line. Default 1 (linear)

    Returns
    ----------
    displayed scatter plot and saved .png of figure

    """
    val = pc.name
    val_dc = dc.name
    fig, ax = plt.subplots(1, 1, figsize=(20, 4), dpi=dpi)
    ax.xaxis.set_tick_params(labelsize=25)
    ax.yaxis.set_tick_params(labelsize=20)

    this_pc = xr.merge((pc / y_norm, gmsl_da), join="left", fill_value=0).sel(
        quantile=0.5
    )
    this_dc = xr.merge((dc / y_norm, gmsl_da), join="left", fill_value=0).sel(
        quantile=0.5
    )

    # Plot median values
    for i, s in enumerate(this_pc.ssp.values):
        ssp_pc = this_pc.sel(ssp=s)
        for m in this_pc.iam.values:
            iam_pc = ssp_pc.sel(iam=m)
            marker = "o"
            xj = -1 * x_iam_jitter

            if m == "IIASA":
                marker = "^"
                xj = x_iam_jitter

            iam_pc.drop_sel(scenario=col_scens).plot.scatter(
                x="gsl_msl05",
                y=val,
                marker=marker,
                edgecolor="none",
                c="#dbdbdb",
                s=30,
                zorder=0.5,
            )
            iam_pc.assign(gsl_msl05=iam_pc.gsl_msl05 + xj).sel(
                scenario=col_scens
            ).plot.scatter(
                x="gsl_msl05",
                y=val,
                marker=marker,
                edgecolor="none",
                c=cvals[i],
                s=100,
                alpha=0.8,
            )

        # Plot trendlines by SSP
        if plot_trend:
            filt = (
                ssp_pc.sel(scenario=col_scens)
                .reset_coords(drop=True)
                .to_dataframe()
                .sort_values("gsl_msl05")
            )

            # calculate equation for trendline and plot
            x = filt["gsl_msl05"]

            z = np.polyfit(x, filt[val], trend_poly)
            p = np.poly1d(z)

            x = [0] + list(x)

            if trend_poly == 1:
                x = [min(x), max(x)]
            y = p(x)
            ax.plot(x, y, c=cvals[i], alpha=0.7, linestyle="dotted", linewidth=1.4)
            coefs = p.coeffs.round(5)
            print(f"{s} Trend Params: {coefs}")

    # Diaz 2016
    this_dc.plot.scatter(
        x="gsl_msl05", y=val_dc, s=120, marker="s", c="black", zorder=1.5
    )

    ax.grid(alpha=0.2, linestyle="--")

    # Set axis lims
    if np.isnan(ymax):
        plt.ylim(ymin, 1.05 * this_pc[val].max().item())
    else:
        plt.ylim(ymin, ymax)

    if np.isnan(xmax):
        plt.xlim(xmin, 1.08 * this_pc.gsl_msl05.max().item())
    else:
        plt.xlim(xmin, xmax)

    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.title(title)

    out_path = DIR_FIGS / str(filename + ".png")
    plt.savefig(out_path, dpi=dpi)

### Global NPV

In [41]:
res_global = res.sum("adm1").load()
d16_global = d16.sum("seg")

res_cc_global = res_cc.sum("adm1").load()
d16_cc_global = d16_cc.sum("seg")

#### SSP2, IIASA vals for Table 3

In [42]:
this_gdp_2100 = global_gdp_2100.sel(ssp=SSP, iam=IAM).item()
this_gdp_npv = global_gdp_npv.sel(ssp=SSP, iam=IAM).item()

this_econ = (
    res_cc_global.sel(ssp=SSP, iam=IAM, quantile=0.5, year=RESENDYR)
    .sum("costtype")[["npv", "costs"]]
    .reset_coords(drop=True)
)
this_econ["npv_frac"] = this_econ.npv / this_gdp_npv * 100
this_econ["costs_frac"] = this_econ.costs / this_gdp_2100 * 100
this_econ[["npv", "costs"]] /= 1e12
this_econ = this_econ.to_dataframe()[
    ["npv", "costs", "npv_frac", "costs_frac"]
].unstack("case")

this_econ_diaz = (
    d16_cc_global.sel(
        quantile=0.5, year=RESENDYR, case=["optimalfixed", "noAdaptation"]
    )
    .sum("costtype")[["npv_consistent", "costs"]]
    .rename(npv_consistent="npv")
    .reset_coords(drop=True)
)
this_econ_diaz["costs_frac"] = this_econ_diaz.costs / (147.6e12 * pl_multiplier) * 100
this_econ_diaz[["npv", "costs"]] /= 1e12
this_econ_diaz = this_econ_diaz.to_dataframe()[["npv", "costs", "costs_frac"]].unstack(
    "case"
)

this_gmsl = gmsl.loc[:, [0.5]]
this_gmsl.columns = pd.MultiIndex.from_tuples(
    [("GMSL", "N/A")], names=this_econ.columns.names
)

tab2 = this_gmsl.join(this_econ.loc[pc_sel_scens], how="right").sort_values(
    ("GMSL", "N/A")
)
tab2.round(2)

GMSL          npv                     costs               \
case                  N/A optimalfixed noAdaptation optimalfixed noAdaptation   
ID         scen                                                                 
Sweet      Low       0.28         0.66         1.21         0.09         0.56   
AR6-Medium SSP1-1.9  0.38         0.80         2.00         0.12         1.03   
           SSP1-2.6  0.44         0.86         2.41         0.15         1.47   
Sweet      IntLow    0.48         0.91         2.46         0.18         1.74   
AR6-Medium SSP2-4.5  0.56         0.98         3.28         0.21         2.46   
           SSP3-7.0  0.68         1.08         4.15         0.27         3.66   
           SSP5-8.5  0.77         1.20         5.24         0.31         4.79   
Sweet      Int       0.98         1.34         6.19         0.42         6.64   
           IntHigh   1.48         1.84        13.62         0.58        13.93   
           High      1.98         2.38        24.61         0.79        24.85   

                        npv_frac                costs_frac               
case                optimalfixed noAdaptation optimalfixed noAdaptation  
ID         scen                                                          
Sweet      Low              0.01         0.03         0.02         0.11  
AR6-Medium SSP1-1.9         0.02         0.04         0.02         0.20  
           SSP1-2.6         0.02         0.05         0.03         0.28  
Sweet      IntLow           0.02         0.05         0.03         0.33  
AR6-Medium SSP2-4.5         0.02         0.07         0.04         0.47  
           SSP3-7.0         0.02         0.09         0.05         0.69  
           SSP5-8.5         0.03         0.11         0.06         0.91  
Sweet      Int              0.03         0.14         0.08         1.26  
           IntHigh          0.04         0.30         0.11         2.64  
           High             0.05         0.54         0.15         4.71

In [43]:
tabc2 = pd.concat(
    (
        this_gmsl.join(this_econ.loc[this_econ_diaz.index], how="right").rename(
            index={"K14": "pyCIAM"}, level="ID"
        ),
        this_gmsl.join(this_econ_diaz, how="right").rename(
            index={"K14": "Diaz 2016"}, level="ID"
        ),
    )
).sort_values(("GMSL", "N/A"))
tabc2.round(2)

GMSL          npv                     costs               \
case                N/A optimalfixed noAdaptation optimalfixed noAdaptation   
ID        scen                                                                
pyCIAM    RCP 2.6  0.48         1.00         2.94         0.14         1.67   
Diaz 2016 RCP 2.6  0.48         1.05         6.84         0.15         1.58   
pyCIAM    RCP 4.5  0.58         1.11         3.80         0.18         2.53   
Diaz 2016 RCP 4.5  0.58         1.17         7.93         0.20         2.04   
pyCIAM    RCP 8.5  0.78         1.31         5.83         0.27         4.68   
Diaz 2016 RCP 8.5  0.78         1.42         9.70         0.29         2.50   

                      npv_frac                costs_frac               
case              optimalfixed noAdaptation optimalfixed noAdaptation  
ID        scen                                                         
pyCIAM    RCP 2.6         0.02         0.06         0.03         0.32  
Diaz 2016 RCP 2.6          NaN          NaN         0.08         0.92  
pyCIAM    RCP 4.5         0.02         0.08         0.03         0.48  
Diaz 2016 RCP 4.5          NaN          NaN         0.12         1.18  
pyCIAM    RCP 8.5         0.03         0.13         0.05         0.89  
Diaz 2016 RCP 8.5          NaN          NaN         0.17         1.45

#### Section 3.2 numbers

##### Ratio of adaptation benefits

In [44]:
this_costs = res_cc_global.npv.sel(quantile=0.5).sum("costtype")
this_ratio = this_costs.sel(case="noAdaptation") / this_costs.sel(case="optimalfixed")
this_ratio.min().item(), this_ratio.max().item()

(1.6171294289037959, 12.204702594162264)

##### Adaptation costs

In [45]:
res_cc_global.npv.sel(
    case="optimalfixed",
    iam=IAM,
    ssp=SSP,
    scenario=("Sweet", "Int"),
    quantile=0.5,
    costtype=["protection", "relocation"],
)

<xarray.DataArray 'npv' (costtype: 2)>
array([7.67755449e+11, 3.14260997e+11])
Coordinates:
    case      <U12 'optimalfixed'
  * costtype  (costtype) <U15 'protection' 'relocation'
    iam       <U5 'IIASA'
    quantile  float64 0.5
    ssp       <U4 'SSP2'
    scenario  object ('Sweet', 'Int')
    ID        <U5 'Sweet'
    scen      <U3 'Int'

##### Range of optimal adaptation NPV across all scenarios

In [46]:
all_global_medians = (
    res_cc_global.npv.sel(quantile=0.5, case="optimalfixed").sum("costtype") / 1e12
)
print(all_global_medians.min().item(), all_global_medians.max().item())

0.6041707135251856 3.4460354380162794


In [47]:
all_global_costs = (
    res_cc_global.costs.sel(year=RESENDYR, quantile=0.5, case="optimalfixed").sum(
        "costtype"
    )
    / 1e12
)
print(all_global_costs.min().item(), all_global_costs.max().item())

0.06989439576864243 1.45956552028656


In [48]:
all_global_costs_k14 = (
    res_cc_global.costs.sel(
        year=RESENDYR,
        quantile=0.5,
        case="optimalfixed",
        scenario=res_cc_global.ID == "K14",
    ).sum("costtype")
    / 1e12
)
print(all_global_costs_k14.min().item(), all_global_costs_k14.max().item())

0.09927264600992203 0.5366625189781189


#### optimal, total

In [ ]:
pc, dc = ds_filt(res_global.npv, d16_global.npv_consistent)

compPlot(
    pc,
    dc,
    filename="global_npv_total_opt",
    ymax=12,
    ymin=-0.2,
)

#### optimal, CC-only

In [ ]:
pc, dc = ds_filt(res_cc_global.npv, d16_cc_global.npv_consistent, cconly=True)

compPlot(
    pc,
    dc,
    filename="global_npv_CC_opt",
    xmin=0.2,
    ymax=4,
    ymin=-0.05,
)

### Wetland Costs end-of-century

#### optimal, total

In [ ]:
pc, dc = ds_filt(
    res_global.costs.sel(year=RESENDYR, drop=True),
    d16_global.costs.sel(year=RESENDYR, drop=True),
    costtypes=["wetland"],
)

compPlot(
    pc,
    dc,
    filename="wetlands_anncosts_total_opt",
    ymax=0.15,
    ymin=-0.005,
)

#### optimal, CC-only 

In [ ]:
pc, dc = ds_filt(
    res_cc_global.costs.sel(year=RESENDYR, drop=True),
    d16_cc_global.costs.sel(year=RESENDYR, drop=True),
    costtypes=["wetland"],
    cconly=True,
)

compPlot(
    pc,
    dc,
    filename="wetlands_anncosts_CC_opt",
    xmin=0.2,
    ymin=-0.01,
    ymax=0.15,
)

### Global Costs end-of-century

#### optimal, total

In [ ]:
pc, dc = ds_filt(
    res_global.costs.sel(year=RESENDYR, drop=True),
    d16_global.costs.sel(year=RESENDYR, drop=True),
)

compPlot(
    pc,
    dc,
    filename="global_anncosts_total_opt",
    ymax=2,
    ymin=-0.1,
)

#### optimal, CC-only 

In [ ]:
pc, dc = ds_filt(
    res_cc_global.costs.sel(year=RESENDYR, drop=True),
    d16_cc_global.costs.sel(year=RESENDYR, drop=True),
    cconly=True,
)

compPlot(
    pc,
    dc,
    filename="global_anncosts_CC_opt",
    xmin=0.2,
    ymin=-0.1,
    ymax=1.8,
)

#### noAdapt, total

In [ ]:
pc, dc = ds_filt(
    res_global.costs.sel(year=RESENDYR, drop=True),
    d16_global.costs.sel(year=RESENDYR, drop=True),
    case="noAdaptation",
)

compPlot(
    pc,
    dc,
    filename="global_anncosts_total_noAdapt",
    ymin=-2,
    ymax=35,
    trend_poly=2,
)

#### noAdapt, CC-only

In [ ]:
pc, dc = ds_filt(
    res_cc_global.costs.sel(year=RESENDYR, drop=True),
    d16_cc_global.costs.sel(year=RESENDYR, drop=True),
    case="noAdaptation",
    cconly=True,
)
p = compPlot(
    pc,
    dc,
    filename="global_anncosts_CC_noAdapt",
    xmin=0.2,
    ymin=-2,
    ymax=35,
    trend_poly=2,
)

### Calc population and K stock btw 0-15m
#### Mid-century (for main text numbers)

In [57]:
pyCIAM_elevs = inputs.elev <= MAXELEV

In [58]:
seg_adm_pop_2050 = inputs.pop_2019.isel(elev=pyCIAM_elevs).sum(
    "elev"
) * inputs.pop_scale.sel(country=inputs.seg_country, year=2050)
print(f'SSP Pops in 2050 (bn): {(seg_adm_pop_2050.sum("seg_adm") / 1e9).values}')

seg_adm_K_2050 = inputs.K_2019.isel(elev=pyCIAM_elevs).sum("elev") * inputs.K_scale.sel(
    country=inputs.seg_country, year=2050
)
print(
    "(IIASA) SSP Ks in 2050 ($Tn): "
    f'{(seg_adm_K_2050.sel(iam="IIASA").sum("seg_adm") / 1e12).values}'
)
print(
    "(OECD) SSP Ks in 2050 ($Tn): "
    f'{(seg_adm_K_2050.sel(iam="OECD").sum("seg_adm") / 1e12).values}'
)

SSP Pops in 2050 (bn): [1.1936475 1.2728753 1.3501482 1.2292199 1.21453  ]
(IIASA) SSP Ks in 2050 ($Tn): [269.92004 256.83463 223.96371 220.11751 305.20374]
(OECD) SSP Ks in 2050 ($Tn): [311.70364 257.9663  217.42883 254.26431 374.58365]


#### Beginning of model time (for figures)

In [59]:
# Get seg (Diaz) population and coastline length totals
seg_pop = (
    d16inputs.pop_2000.sum("elev") * d16inputs.pop_scale.sel(year=POPWTYR, drop=True)
).rename("pop")
seg_len = d16inputs.length

# Get seg_adm (pyCIAM) population and coastline length totals
# all ssps will have same pop for years <=2010
seg_pop_pc = (
    (
        inputs.isel(elev=pyCIAM_elevs).pop_2019.sum("elev")
        * inputs.pop_scale.sel(year=POPWTYR, ssp=SSP, drop=True).sel(
            country=inputs.seg_country, drop=True
        )
    )
    .rename("pop")
    .groupby("seg")
    .sum()
)
seg_len_pc = inputs.length.groupby("seg").sum()

### Optimal Retreat Strategy

In [60]:
# Dictionary of what numbers mean which case
case_dict = literal_eval(
    open_zarr(PATH_OUTPUTS)
    .optimal_case.attrs["Description"]
    .lstrip("Coded as follows: ")
)
protect_cases = [v for k, v in case_dict.items() if k.startswith("protect")]
retreat_cases = [v for k, v in case_dict.items() if k.startswith("retreat")]
noadapt_case = case_dict["noAdaptation"]

#### Gather data

In [61]:
# Get dataframes of optimal strategy by scenario for Diaz config
dc = xr.merge(
    (
        d16.sel(quantile=0.5).optimal_case.sel(scenario=d16.scen != "NCC"),
        seg_pop,
        seg_len,
    )
)

# Get dataframes of optimal strategy by scenario for pyCIAM
pc = xr.merge(
    (
        res.sel(quantile=0.5).optimal_case.sel(scenario=res.scen != "NCC"),
        seg_pop_pc,
        seg_len_pc,
    )
).load()

# Get percentages
strategy_d16 = xr.where(
    dc.optimal_case.isin(protect_cases),
    "protect",
    xr.where(dc.optimal_case.isin(retreat_cases), "retreat", "noAdapt"),
).to_series()

strat_by_exposure_d16 = (
    strategy_d16.to_frame()
    .join(dc[["pop", "length"]].reset_coords(drop=True).to_dataframe(), on="seg")
    .groupby(["ID", "scen", "optimal_case"])
    .sum()
)
strat_by_exposure_d16 /= strat_by_exposure_d16.groupby(["ID", "scen"]).sum()

strategy_pc = xr.where(
    pc.optimal_case.load().isin(protect_cases),
    "protect",
    xr.where(pc.optimal_case.isin(retreat_cases), "retreat", "noAdapt"),
).to_series()

strat_by_exposure_pc = (
    strategy_pc.to_frame()
    .join(pc[["pop", "length"]].reset_coords(drop=True).to_dataframe(), on="seg")
    .groupby(["ID", "scen", "ssp", "iam", "optimal_case"])
    .sum()
)
strat_by_exposure_pc /= strat_by_exposure_pc.groupby(["ID", "scen", "ssp", "iam"]).sum()

# convert to xarray and to percent
strat_by_exposure_d16 = (
    strat_by_exposure_d16.to_xarray().stack(scenario=["ID", "scen"]).dropna("scenario")
    * 100
)
strat_by_exposure_pc = (
    strat_by_exposure_pc.to_xarray().stack(scenario=["ID", "scen"]).dropna("scenario")
    * 100
)

#### Plot

In [ ]:
for v in strat_by_exposure_pc.data_vars:
    for strat in strat_by_exposure_pc.optimal_case.values:
        print(v, strat)
        compPlot(
            strat_by_exposure_pc[v].sel(optimal_case=strat, drop=True),
            strat_by_exposure_d16[v].sel(optimal_case=strat, drop=True),
            filename=f"optcase_{strat}_{v}",
            xmin=0.20,
            ymin=-5,
            ymax=100,
            plot_trend=False,
            y_norm=1,
        )

## Prepare Data for Maps

### Compute Median Costs in 2100

In [63]:
adm1_costs = res_cc.costs.sel(year=RESENDYR, drop=True).load()

### Import SHPs for Maps

In [64]:
adm1 = (
    read_shapefile(PATH_GADM, layer="ADM_1")
    .rename(columns={"GID_1": "adm1", "GID_0": "country"})
    .set_index("adm1")[["country", "geometry"]]
)

# correct Ghana adm1 code typo in GADM level 1 labels
adm1.index = (
    adm1.index.str.replace("GHA", "GHA.")
    .to_series()
    .replace("CHN.HKG", "HKG")
    .replace("CHN.MAC", "MAC")
    .values
)

# Country
ctys = (
    read_shapefile(PATH_GADM, layer="ADM_0")
    .rename(columns={"GID_0": "country"})
    .set_index("country")
)

# Add single adm1 countries (e.g. Aruba) to adm1 file
to_append = ctys.loc[adm1_costs.adm1[~adm1_costs.adm1.isin(adm1.index)].values].drop(
    columns="COUNTRY"
)
to_append = to_append.assign(country=to_append.index).rename_axis(index=None)
adm1 = pd.concat((adm1, to_append))

# add HKG and MAC to ctys
ctys = pd.concat(
    (
        ctys,
        adm1.loc[["HKG", "MAC"]]
        .assign(COUNTRY=["Hong Kong", "Macao"])
        .rename_axis(index="country")
        .drop(columns="country"),
    )
)

# Import borders and coastlines SHPs
borders = read_shapefile(PATH_BORDERS)
clines = read_shapefile(PATH_COASTLINES)

### Mapping Functions

In [65]:
# CIL Color Map (Reds)
colors = [
    "#fff3d7",
    "#fbc885",
    "#f39b65",
    "#ea744b",
    "#e45d3d",
    "#db4231",
    "#d51d26",
]
edgecolor = "#808080"
cil_cmap = LinearSegmentedColormap.from_list("mycm", colors, N=100)
crs = ccrs.Robinson()


def costs_poly_plot(
    polyshp,
    val_col="costs",
    filename="mymap",
    vmin=0,
    vmax=1,
    cm=cil_cmap,
    symlog=True,
    linthresh=1,
    linscale=1,
    dpi=300,
):
    """creates map of results values for given scenario for geographic boundaries (e.g.
    national, ADM1)

    Parameters
    ----------
    polyshp : gdp.GeoDataFrame
        gdf with joined results variables of polygons to plot (e.g. national or ADM1
        boundaries)
    val_col : str
        name of variable to plot
    filename : str
        name of saved map file .png
    vmin, vmax : int or float
        range of color scale variable (z values) on map
    cm : int
        color map for plot
    symlog : bool
        enable symmetric log for color scale
    linthresh : int or float
        The range within which the plot is linear
    linscale : int or float
        This allows the linear range to be stretched relative to the logarithmic
        range.
    """
    fig, ax = plt.subplots(figsize=(40, 20), subplot_kw={"projection": crs}, dpi=dpi)
    ax.axis("off")

    if symlog:
        kwargs = {
            "norm": mcols.SymLogNorm(linthresh, linscale=linscale, vmin=vmin, vmax=vmax)
        }
    else:
        kwargs = {"vmin": vmin, "vmax": vmax}
    polyshp.to_crs(crs).plot(
        column=val_col,
        ax=ax,
        legend=True,
        cmap=cm,
        markersize=120,
        edgecolor=edgecolor,
        linewidth=0,
        alpha=0.8,
        zorder=10,
        **kwargs
    )

    if filename is not None:
        fig.savefig(DIR_FIGS / str(filename + ".png"), dpi=dpi)
    return ax

## Generate Maps
### Costs in 2100

In [ ]:
# Total Costs in 2100 (USD)
cost_map_2100 = adm1.join(
    adm1_costs.sel(
        case="optimalfixed",
        scenario=("AR6-Medium", "SSP2-4.5"),
        quantile=0.5,
        ssp=SSP,
        iam=IAM,
        drop=True,
    )
    .sum("costtype")
    .to_dataframe(),
    how="right",
)

costs_poly_plot(
    cost_map_2100,
    filename="map_adm1_global_costs_CC_opt",
    vmin=1e7,
    vmax=1e10,
)

In [ ]:
# Benefits of Adaptation in 2100 [USD]
tot_costs = adm1_costs.sel(
    scenario=("AR6-Medium", "SSP2-4.5"),
    quantile=0.5,
    ssp=SSP,
    iam=IAM,
    drop=True,
).sum("costtype")

ben_adapt = adm1.join(
    (
        tot_costs.sel(case="noAdaptation", drop=True)
        - tot_costs.sel(case="optimalfixed", drop=True)
    ).to_series(),
    how="right",
)
costs_poly_plot(
    ben_adapt,
    filename="map_adm1_global_benAdapt_CC_opt",
    vmin=1e7,
    vmax=1e11,
    cm="Greens",
)

### Optimal Adaptation Strategy

In [68]:
adapt_dict = {
    "RETREAT": retreat_cases,
    "PROTECT": protect_cases,
    "NOADAPT": noadapt_case,
}

scen_res = res.sel(
    quantile=0.5,
    ssp=SSP,
    iam=IAM,
    scenario=SCEN,
    case=["noAdaptation", "optimalfixed"],
    drop=True,
).load()

adapt = xr.merge(
    (
        scen_res.optimal_case,
        inputs[["seg_lat", "seg_lon"]]
        .rename(seg_lat="lat", seg_lon="lon")
        .load()
        .groupby(inputs.seg)
        .first(),
    )
)
adapt = (
    gpd.GeoSeries.from_xy(adapt.lon.to_series(), adapt.lat.to_series(), crs="EPSG:4326")
    .to_frame(name="geometry")
    .join(adapt.optimal_case.to_series())
)


def adapt_plot(
    gdf,
    val_col="optimal_case",
    adapt_type="RETREAT",
    filename="myplot",
    vmin=0,
    vmax=300,
    alpha=0.8,
    msize=120,
    cm="YlGnBu",
    dpi=300,
):
    """creates map of results values for given scenario for geographic boundaries (e.g.
    national, ADM1)

    ----------
    gdf : gdp.GeoDataFrame
        gdf with optimal adaptation results joined with segment centroid SHP
    val_col : str
        name of variable to plot
    adapt_type : str
        which type of adaptation strategies to plot (PROTECT, RETREAT, NOADAPT)
    filename : str
        name of saved map file .png
    vmin, vmax : int or float
        range of color scale variable (z values) on map
    alpha : float
        opacity of plotted points
    msize : int
        point marker size
    cm : int
        color map for plot
    """
    fig, ax = plt.subplots(figsize=(40, 20), subplot_kw={"projection": crs}, dpi=dpi)
    ax.axis("off")

    # International Borders
    borders.to_crs(crs).plot(
        ax=ax, facecolor="none", edgecolor=edgecolor, linewidth=1, alpha=0.5, zorder=5
    )

    # Coastlines
    clines.to_crs(crs).plot(
        ax=ax, edgecolor=edgecolor, linewidth=1, alpha=0.5, zorder=0
    )

    filters = adapt_dict[adapt_type]
    if isinstance(filters, int):
        filters = [filters]

    gdf[gdf[val_col].isin(filters)].to_crs(crs).plot(
        column=val_col,
        ax=ax,
        legend=True,
        cmap=cm,
        vmin=vmin,
        vmax=vmax,
        markersize=msize,
        edgecolor=edgecolor,
        linewidth=0,
        alpha=alpha,
        zorder=10,
    )

    plt.savefig(DIR_FIGS / str(filename + ".png"), dpi=dpi)
    return ax

In [ ]:
# r_colors = ["yellow", "skyblue", "royalblue", "darkblue", "black"]
r_colors = ["#f2f200", "#40d3e0", "dodgerblue", "blue", "midnightblue"]
p_colors = ["#40d3e0", "dodgerblue", "blue", "midnightblue"]
na_colors = ["grey", "grey"]

for adapt_type, colors, vmin, vmax in [
    ("RETREAT", r_colors, 5, 9),
    ("PROTECT", p_colors, 1, 4),
    ("NOADAPT", na_colors, 0, 0),
]:
    adapt_plot(
        adapt,
        adapt_type=adapt_type,
        filename="optAdapt_map_retreat",
        vmin=vmin,
        vmax=vmax,
        alpha=1,
        msize=70,
        cm=LinearSegmentedColormap.from_list("cm", colors, N=len(colors)),
    )

In [70]:
client.cluster.close(), client.close()

(None, None)